In [ ]:
import numpy as np
import pandas as pd
import datatable as dt
import gc
from numba import njit

from scipy.optimize import minimize
from sklearn.neighbors import KernelDensity

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Load data

In [ ]:
train_df = dt.fread('../input/jane-street-market-prediction/train.csv').to_pandas()

# Denoising Correlation matrix

In [ ]:
# Snippets from Marco Lopez de Prado, 2020

def mpPDF(var,q,pts):
    # Marcenko-Pastur pdf
    # q=T/N
    eMin, eMax = var*(1-(1./q)**.5)**2, var*(1+(1./q)**.5)**2
    eVal = np.linspace(eMin,eMax,pts)
    pdf = q/(2*np.pi*var*eVal)*((eMax-eVal)*(eVal-eMin))**.5
    pdf = pd.Series(pdf.reshape(-1,), index=eVal.reshape(-1,))
    return pdf


def getPCA(matrix):
    # Get eVal,eVec from a Hermitian matrix
    eVal,eVec = np.linalg.eigh(matrix)
    indices=eVal.argsort()[::-1] # arguments for sorting eVal desc
    eVal,eVec=eVal[indices],eVec[:,indices]
    eVal=np.diagflat(eVal)
    return eVal,eVec

def fitKDE(obs,bWidth=.25,kernel='gaussian',x=None):
    # Fit kernel to a series of obs, and derive the prob of obs
    # x is the array of values on which the fit KDE will be evaluated
    if len(obs.shape)==1:
        obs=obs.reshape(-1,1)
    kde=KernelDensity(kernel=kernel,bandwidth=bWidth).fit(obs)
    if x is None:
        x=np.unique(obs).reshape(-1,)
    if len(x.shape)==1:
        x=x.reshape(-1,1)
    logProb=kde.score_samples(x) # log(density)
    pdf=pd.Series(np.exp(logProb),index=x.flatten())
    return pdf

def cov2corr(cov):
    # Derive the correlation matrix from a covariance matrix
    std=np.sqrt(np.diag(cov))
    corr=cov/np.outer(std,std)
    corr[corr<-1],corr[corr>1]=-1,1 # numerical error
    return corr

def errPDFs(var,eVal,q,bWidth,pts=1000):
    # Fit error
    pdf0=mpPDF(var,q,pts) # theoretical pdf
    pdf1=fitKDE(eVal,bWidth,x=pdf0.index.values) # empirical pdf
    sse=np.sum((pdf1-pdf0)**2)
    return sse

def findMaxEval(eVal,q,bWidth):
    # Find max random eVal by fitting Marcenko’s dist
    out=minimize(lambda *x:errPDFs(*x),.5,args=(eVal,q,bWidth),
    bounds=((1E-5,1-1E-5),))
    if out['success']:
        var=out['x'][0]
    else:
        var=1
    eMax=var*(1+(1./q)**.5)**2
    return eMax,var

def denoisedCorr(eVal,eVec,nFacts):
    # Remove noise from corr by fixing random eigenvalues
    eVal_=np.diag(eVal).copy()
    eVal_[nFacts:]=eVal_[nFacts:].sum()/float(eVal_.shape[0] - nFacts)
    eVal_=np.diag(eVal_)
    corr1=np.dot(eVec,eVal_).dot(eVec.T)
    corr1=cov2corr(corr1)
    return corr1

def denoisedCorr2(eVal,eVec,nFacts,alpha=0):
    # Remove noise from corr through targeted shrinkage
    eValL,eVecL=eVal[:nFacts,:nFacts],eVec[:,:nFacts]
    eValR,eVecR=eVal[nFacts:,nFacts:],eVec[:,nFacts:]
    corr0=np.dot(eVecL,eValL).dot(eVecL.T)
    corr1=np.dot(eVecR,eValR).dot(eVecR.T)
    corr2=corr0+alpha*corr1+(1-alpha)*np.diag(np.diag(corr1))
    return corr2

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

@njit
def fillna_npwhere_njit(array, values):
    if np.isnan(array.sum()):
        array = np.where(np.isnan(array), values, array)
    return array

class RMTDenoising(BaseEstimator, TransformerMixin):
    
    def __init__(self, bWidth=.01, alpha=.5, feature_0=True, sample=0.3, seed=2021):
        self.bWidth = bWidth
        self.alpha = alpha
        self.feature_0 = feature_0
        self.sample = sample
        self.seed = seed
    
    def denoise(self, X):
        sample = X.sample(frac=self.sample, random_state=self.seed)
        q = X.shape[0] / X.shape[1]
        cov = sample.cov().values
        corr0 = cov2corr(cov)

        eVal0, eVec0 = getPCA(corr0)
        eMax0, var0 = findMaxEval(np.diag(eVal0), q, bWidth=self.bWidth)
        nFacts0 = eVal0.shape[0] - np.diag(eVal0)[::-1].searchsorted(eMax0)
        corr1 = denoisedCorr2(eVal0,eVec0,nFacts0,alpha=self.alpha)
        eVal1, eVec1 = getPCA(corr1)
        #result = np.hstack((np.diag(eVal1), var0))
        #name = [f'eigen_{i+1}' for i in range(len(eVal1))] + ['var_explained']
        return eVec1[:, :nFacts0]
    
    def fit(self, X, y=None):
        if self.feature_0:
            self.cols_ = [c for c in X.columns if c != 'feature_0']
        else:
            self.cols_ = list(X.columns)
        X_ = X[self.cols_]
        self.W_ = self.denoise(X_)
        self.dim_W_ = self.W_.shape[1]
        return self
    
    def transform(self, X, y=None):
        X_ = X.copy()
        names = [f'proj_{i}' for i in range(self.dim_W_)]
        projection = pd.DataFrame(fillna_npwhere_njit(X_[self.cols_].values, 0).dot(self.W_), columns=names)
        if self.feature_0:
            projection['feature_0'] = X['feature_0']
        return projection


# Modelling

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

In [ ]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score

In [ ]:
# target
th = 0
train_df['action'] = (train_df.resp * train_df.weight) > 0

# features
feature_cols = [c for c in train_df.columns if c.startswith('feature_')]

X = train_df[feature_cols]
y = train_df['action']
groups = train_df.date.values
eps = 1e-3
wgts = (train_df['weight'] + eps).values # proxy for risk

del train_df
gc.collect()

In [ ]:
# params
params = dict(
    n_estimators=150,
    learning_rate=.2,
    gamma=1.2,
    max_depth=3,
    reg_lambda=1.8,
    subsample=.63,
    colsample_bytree=.8,
    random_state=2021,
    tree_method='gpu_hist',
    min_child_weight=0.03,
    n_jobs=-1,
    predictor='gpu_predictor'
)

model = Pipeline([
    ('denoise', RMTDenoising(sample=0.8)),
    ('clf', XGBClassifier(**params))
], verbose=True)

## Cross validation

In [ ]:
# cv params
FOLDS = 5
cv = PurgedGroupTimeSeriesSplit(n_splits=FOLDS, 
                                group_gap=10,
                                #max_train_group_size=10000,
                                max_test_group_size=10000)

# cross_validate
scores = cross_val_score(
    model, X, y, cv=cv, groups=groups, 
    fit_params=dict(clf__sample_weight=wgts),
    scoring='roc_auc', verbose=10)

print(np.mean(scores), np.std(scores))
print(scores)

In [ ]:
# fit
model.fit(X, y, clf__sample_weight=wgts)

# Submission

In [ ]:
import janestreet
from tqdm import tqdm

env = janestreet.make_env()

In [ ]:
for (test_df, pred_df) in tqdm(env.iter_test()):
    if test_df['weight'].item() > 0:
        x_tt = test_df[feature_cols]
        pred = model.predict(x_tt)
        pred_df.action = pred
    else:
        pred_df.action = 0
    pred_df.action = pred_df.action.astype('int')
    env.predict(pred_df)